# Necessary Tools and Services

In [3]:
! pip install pandas
! pip install matplotlib
! pip install numpy
! pip install seaborn
! pip install unzip
! pip install gensim
! pip install nltk
! pip install wordcloud
! pip install spacy
! pip install spacy_download
! pip install pyLDAvis
! pip install PyStemmer

! python3 -m spacy download en

  Using cached seaborn-0.13.0-py3-none-any.whl (294 kB)
  Using cached unzip-1.0.0-py3-none-any.whl
  Using cached wordcloud-1.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (455 kB)
  Using cached spacy-3.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 918.3 kB/s eta 0:00:00 0:00:01
  Using cached cymem-2.0.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (46 kB)
  Using cached srsly-2.4.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (493 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached wasabi-1.1.2-py3-none-any.whl (27 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl (22 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached thinc-8.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (920 kB)
  Using cached murmurhash-1.0.10-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/nextflow_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Body"]]
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

72


In [3]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['This is really a Nextflow question. I have a process that produces a number '
 'of files, some will be empty. I need to detect which ones and the next '
 'process should skip processing them. Looked straightforward, but the code: '
 'process demuxByPrimers { publishDir params.outdir input: file productFile '
 'from products output: file ITS*.fastq into primers mode flatten when: '
 'productFile.size() &gt; 0 script: println "Processing ${productFile}" } does '
 'not work well. Input files in the work directory are symbolic links and thus '
 'they are not empty... Any ideas on how to skip processing empty files? ',
 'I have trying to create a conditional pipeline NextFlow. For example, '
 'Process A outputs a value to a channel. If the value is 1, then run X, '
 'otherwise run Y. Heres what I am trying to do: initialData = 2 receiver1 = '
 '"EMPTY" receiver2 = "EMPTY" receiver3 = "" process A { input: val '
 'initialData output: val initialData into trigger  echo 10  } process foo { '

# Tokenize words and Clean-up text

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['this', 'is', 'really', 'nextflow', 'question', 'have', 'process', 'that', 'produces', 'number', 'of', 'files', 'some', 'will', 'be', 'empty', 'need', 'to', 'detect', 'which', 'ones', 'and', 'the', 'next', 'process', 'should', 'skip', 'processing', 'them', 'looked', 'straightforward', 'but', 'the', 'code', 'process', 'demuxbyprimers', 'publishdir', 'params', 'outdir', 'input', 'file', 'productfile', 'from', 'products', 'output', 'file', 'its', 'fastq', 'into', 'primers', 'mode', 'flatten', 'when', 'productfile', 'size', 'gt', 'script', 'println', 'processing', 'productfile', 'does', 'not', 'work', 'well', 'input', 'files', 'in', 'the', 'work', 'directory', 'are', 'symbolic', 'links', 'and', 'thus', 'they', 'are', 'not', 'empty', 'any', 'ideas', 'on', 'how', 'to', 'skip', 'processing', 'empty', 'files']]


In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['really', 'nextflow', 'question', 'process', 'produce', 'number', 'file', 'empty', 'need', 'detect', 'one', 'next', 'process', 'skip', 'processing', 'look', 'straightforward', 'code', 'process', 'demuxbyprimer', 'input', 'file', 'productfile', 'product', 'output', 'file', 'fastq', 'primer', 'mode', 'flatten', 'productfile', 'size', 'script', 'println', 'processing', 'productfile', 'work', 'well', 'input', 'file', 'work', 'directory', 'symbolic', 'link', 'thus', 'empty', 'idea', 'skip', 'process', 'empty', 'file']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 4), (22, 2), (23, 1), (24, 1), (25, 3), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2)]]


In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('code', 1),
  ('demuxbyprimer', 1),
  ('detect', 1),
  ('directory', 1),
  ('empty', 3),
  ('fastq', 1),
  ('file', 5),
  ('flatten', 1),
  ('idea', 1),
  ('input', 2),
  ('link', 1),
  ('look', 1),
  ('mode', 1),
  ('need', 1),
  ('next', 1),
  ('nextflow', 1),
  ('number', 1),
  ('one', 1),
  ('output', 1),
  ('primer', 1),
  ('println', 1),
  ('process', 4),
  ('processing', 2),
  ('produce', 1),
  ('product', 1),
  ('productfile', 3),
  ('question', 1),
  ('really', 1),
  ('script', 1),
  ('size', 1),
  ('skip', 2),
  ('straightforward', 1),
  ('symbolic', 1),
  ('thus', 1),
  ('well', 1),
  ('work', 2)]]

In [13]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [12]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [05:49<00:00,  1.55it/s]


In [14]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
537,100% Corpus,10,asymmetric,0.61,0.495722
523,100% Corpus,10,0.61,0.9099999999999999,0.485787
538,100% Corpus,10,asymmetric,0.9099999999999999,0.482608
418,100% Corpus,6,asymmetric,0.9099999999999999,0.468945
518,100% Corpus,10,0.31,0.9099999999999999,0.466430
...,...,...,...,...,...
353,100% Corpus,4,symmetric,0.9099999999999999,0.341191
338,100% Corpus,4,0.31,0.9099999999999999,0.341191
462,100% Corpus,8,0.61,0.61,0.339424
432,100% Corpus,7,0.61,0.61,0.338754


In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=100,
                                           alpha=0.61,
                                           eta=0.61)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -6.267277436247078

Coherence Score:  0.45240046862634303


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.082283 -0.052369       1        1  51.122273
1     -0.099088 -0.040693       2        1  20.648357
2      0.078378  0.042922       3        1  18.907484
0     -0.061573  0.050139       4        1   9.321886, topic_info=          Term        Freq       Total Category  logprob  loglift
86       debug  100.000000  100.000000  Default  30.0000  30.0000
117       main  102.000000  102.000000  Default  29.0000  29.0000
15    nextflow  195.000000  195.000000  Default  28.0000  28.0000
141    session   42.000000   42.000000  Default  27.0000  27.0000
21     process  204.000000  204.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
12        mode    2.179401    9.055800   Topic4  -5.7926   0.9484
189      check    2.187255   11.800261   Topic4  -5.7890   0.6873
115        log    2.178294   17.000202   Topic4  -5.7931   0.3181
124         nf    2.173810   26.894087   Topic4  -5.7951  -0.1426
1071     class    2.158033   12.316183   Topic4  -5.8024   0.6311

[263 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1064      4  0.402217         abort
419       1  0.138887     accession
419       3  0.833322     accession
63        2  0.803869  actor_thread
354       1  0.876257          also
...     ...       ...           ...
157       1  0.635391      workflow
157       2  0.074752      workflow
157       3  0.261632      workflow
418       1  0.831122         write
418       3  0.103890         write

[379 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1])

In [12]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.044*"nextflow" + 0.017*"main" + 0.017*"debug" + 0.015*"avr" + '
  '0.015*"session" + 0.013*"plugin" + 0.012*"groovy" + 0.011*"oct_main" + '
  '0.010*"run" + 0.010*"script"'),
 (1,
  '0.055*"debug" + 0.047*"main" + 0.044*"nextflow" + 0.020*"session" + '
  '0.019*"process" + 0.013*"executor" + 0.010*"file" + 0.009*"final" + '
  '0.008*"var_singularity" + 0.008*"processor_processfactory"'),
 (2,
  '0.041*"process" + 0.017*"output" + 0.013*"param" + 0.013*"file" + '
  '0.011*"path" + 0.011*"nextflow" + 0.010*"input" + 0.010*"use" + 0.008*"run" '
  '+ 0.008*"script"'),
 (3,
  '0.043*"file" + 0.029*"process" + 0.023*"param" + 0.020*"nextflow" + '
  '0.018*"output" + 0.015*"error" + 0.015*"input" + 0.013*"run" + 0.012*"path" '
  '+ 0.012*"use"')]


In [13]:
num_topics = 10
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head(3))
df.to_csv('../dataset/AssignedTopicNextflow.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                         Title  \
0  47401518             Nextflow: Is an input file empty?   
1  50443545              Conditional Pipeline in Nextflow   
2  53277369  splitCsv then map a list of URLs in Nextflow   

                                                Body  RatingsSentiCR  \
0  This is really a Nextflow question. I have a p...              -1   
1  I have trying to create a conditional pipeline...              -1   
2  I am trying to take the GIAB data index files ...              -1   

   RatingsGPT35  RatingsGPTFineTuned  \
0            -1                 -1.0   
1             1                 -1.0   
2            -1                 -1.0   

                                              merged  topic  
0  This is really a Nextflow question. I have a p...      3  
1  I have trying to create a conditional pipeline...      2  
2  I am trying to take the GIAB data index files ...      1  
topic:  0 total data:  4
topic:  1 total data:  5
topic: 